In [1]:
from faker import Faker
import pandas as pd

faker = Faker('pt_BR')

In [53]:
USER_AMOUNT = 3
user_names = [faker.name() for _ in range(USER_AMOUNT)]
emails = [faker.company_email() for _ in range(USER_AMOUNT)]
password = "$2a$10$6IYpRegV7yL4.HYfYSOC3uVyg7v8BFpRZ1uen1I0NQScyi7UHd50W"

administrators = pd.DataFrame({
    "email": emails,
    "nome": user_names,
    "password": [password for _ in range(USER_AMOUNT)],
}, columns={"email": str, "nome": str, "password": str})
administrators.head()

,email,nome,password
0,levi41@alves.org,Fernanda Moraes,$2a$10$6IYpRegV7yL4.HYfYSOC3uVyg7v8BFpRZ1uen1I...
1,gustavoda-luz@dias.com,Dra. Heloísa Rezende,$2a$10$6IYpRegV7yL4.HYfYSOC3uVyg7v8BFpRZ1uen1I...
2,joao-miguel91@viana.br,Caroline Melo,$2a$10$6IYpRegV7yL4.HYfYSOC3uVyg7v8BFpRZ1uen1I...


In [54]:
CONDOMINIUMS_AMOUNT = [faker.pyint(1, 10) for _ in range(USER_AMOUNT)]
TOTAL_APARTMENTS = sum(CONDOMINIUMS_AMOUNT)

condominiums_emails = []
for x in range(USER_AMOUNT):
    condominiums_emails += [emails[x]] * CONDOMINIUMS_AMOUNT[x]

condominiums = pd.DataFrame({
    "administradorEmail": condominiums_emails,
    "nomeCondominio": [faker.company() for _ in range(TOTAL_APARTMENTS)],
}, columns={"nomeCondominio": str, "administradorEmail": str})

condominiums.head()

,nomeCondominio,administradorEmail
0,Pinto Ltda.,levi41@alves.org
1,Moraes Lopes S.A.,levi41@alves.org
2,Silveira Alves e Filhos,levi41@alves.org
3,Carvalho,levi41@alves.org
4,Teixeira,levi41@alves.org


In [55]:
debtors = pd.DataFrame(columns={"cpf": str, "nome": str, "valorDivida": float,
                                "mensalidadesAtrasadas": int,
                                "nomeCondominio": str})

for index, row in condominiums.iterrows():
  DEBTORS_AMOUNT = faker.pyint(1, 30)
  debtors = pd.concat([debtors, pd.DataFrame({
    "cpf": [faker.cpf() for _ in range(DEBTORS_AMOUNT)],
    "nome": [faker.name() for _ in range(DEBTORS_AMOUNT)],
    "valorDivida": [faker.pydecimal(left_digits=4, right_digits=2,
                                      positive=True, min_value=250
                          ) for _ in range(DEBTORS_AMOUNT)],
    "mensalidadesAtrasadas": [faker.pyint(1, 20) for _ in
                              range(DEBTORS_AMOUNT)],
    "nomeCondominio": row["nomeCondominio"],
    "emailAdministrador": row["administradorEmail"]
  })], ignore_index=True)

debtors

,cpf,nome,valorDivida,mensalidadesAtrasadas,nomeCondominio,emailAdministrador
0,421.305.786-08,Nicolas da Cruz,9444.73,9,Pinto Ltda.,levi41@alves.org
1,972.350.186-40,Cauê Souza,283.82,19,Pinto Ltda.,levi41@alves.org
2,935.748.260-10,Enrico Sales,6009.95,18,Pinto Ltda.,levi41@alves.org
3,641.730.592-99,Vitória da Costa,5085.85,2,Pinto Ltda.,levi41@alves.org
4,264.579.138-73,Bryan Gomes,9738.08,6,Pinto Ltda.,levi41@alves.org
...,...,...,...,...,...,...
276,012.865.349-33,Mariana Silveira,1353.58,4,Barbosa,joao-miguel91@viana.br
277,359.427.168-46,Maria Alice Souza,8763.25,20,Barbosa,joao-miguel91@viana.br
278,031.265.879-68,Fernanda das Neves,5275.60,16,Barbosa,joao-miguel91@viana.br
279,689.401.273-31,Srta. Natália Mendes,2974.24,20,Barbosa,joao-miguel91@viana.br


In [56]:
administrators.to_csv("./administrators.csv", index=False)
debtors.to_csv("./debtors.csv", index=False)